Library Imports

In [147]:
from braket.circuits.noise_model import (
    GateCriteria,
    NoiseModel,
    ObservableCriteria,
)
from braket.circuits import Circuit, Observable, Gate
from braket.circuits.noises import (
    BitFlip,
    Depolarizing,
    TwoQubitDepolarizing,
)
from braket.devices import LocalSimulator
import numpy as np
import math

Noise Model for Testing

In [148]:
def noise_model():
    rng = np.random.default_rng()
    m = NoiseModel()
    
    two_q_depo_mu = 1 - 0.9311
    two_q_depo_sigma = 0.005
    bf_mu = 1 - 0.99752
    bf_sigma = 0.0015
    one_q_depo_mu = 1 - 0.9981
    one_q_depo_sigma = 0.00017
    for qi in range(11):
        z_bf_prob = bf_mu + bf_sigma * rng.standard_normal()
        z_bf_prob = 0.0 if z_bf_prob < 0.0 else z_bf_prob
        
        bf_prob = bf_mu + bf_sigma * rng.standard_normal()
        bf_prob = 0.0 if bf_prob < 0.0 else bf_prob
        
        one_q_depo_prob = one_q_depo_mu + one_q_depo_sigma * rng.standard_normal()
        one_q_depo_prob = 0.0 if one_q_depo_prob < 0.0 else one_q_depo_prob
        
        m.add_noise(BitFlip(z_bf_prob), ObservableCriteria(observables=Observable.Z, qubits=qi))
        #m.add_noise(BitFlip(bf_prob), ObservableCriteria(qubits=qi))
        
        m.add_noise(Depolarizing(one_q_depo_prob), GateCriteria(qubits=qi))
        for qj in range(11):
            if not qj == qi:
                two_q_depo_prob = two_q_depo_mu + two_q_depo_sigma * rng.standard_normal()
                two_q_depo_prob = 0.0 if two_q_depo_prob < 0.0 else two_q_depo_prob
                
                m.add_noise(TwoQubitDepolarizing(two_q_depo_prob), GateCriteria(gates=[Gate.CNot, Gate.Swap, Gate.CPhaseShift], qubits=[qi, qj]))
    return m

Global Parameters

In [149]:
# select the local noise simulator
device = LocalSimulator('braket_dm')
noise = noise_model()
n = 11

Single Qubit Fidelity Measurement

In [150]:
def create_hadamard_circuit(gate_num = 2):
    c = Circuit()
    for qubit in range(n):
        for _ in range(gate_num):
            c.h(qubit)
    c = noise.apply(c)
    return c

def calc_single_qubit_fidelity(had_cnt = 2):
    c = create_hadamard_circuit(had_cnt)
    shotnum = 10000
    task = device.run(c, shots = shotnum)
    result = task.result()
    measurement = result.measurement_counts
    measurement = {int(state[::-1], 2) : cnt for state, cnt in measurement.items()}
    return [(sum(cnt for state, cnt in measurement.items() if not (state&(2**qubit)))/shotnum)**0.5 for qubit in range(n)]

In [151]:
calc_single_qubit_fidelity()

[0.9988993943335835,
 0.9987992791347018,
 0.9986490875177326,
 0.9985489472229191,
 0.9984988733093293,
 0.9986490875177326,
 0.9990995946350895,
 0.9987992791347018,
 0.9992997548283498,
 0.9989494481704267,
 0.9983987179478948]

Two Qubit Fidelity Measurement

In [152]:
# Round Robin Creation

rownum = n-1+(n&1)
colnum = (n+1)//2

table = [[] for _ in range(rownum)]

cur = 0

for row in range(rownum):
    for col in range(colnum):
        table[row].append([cur])
        cur += 1
        cur %= n+1-(n&1)

for row in range(rownum):
    place_row = (row-1)%rownum
    for col in range(colnum-1, -1, -1):
        place_col = colnum-1-col
        table[place_row][place_col].append(table[row][col][0])
    if not n&1:
        table[place_row][place_col][1] = n-1

for row in range(rownum):
    cur = []
    for col in range(colnum):
        cur.append(table[row][col][::-1])
    table.append(cur)

print (*table, sep='\n')


[[0, 0], [1, 10], [2, 9], [3, 8], [4, 7], [5, 6]]
[[6, 6], [7, 5], [8, 4], [9, 3], [10, 2], [0, 1]]
[[1, 1], [2, 0], [3, 10], [4, 9], [5, 8], [6, 7]]
[[7, 7], [8, 6], [9, 5], [10, 4], [0, 3], [1, 2]]
[[2, 2], [3, 1], [4, 0], [5, 10], [6, 9], [7, 8]]
[[8, 8], [9, 7], [10, 6], [0, 5], [1, 4], [2, 3]]
[[3, 3], [4, 2], [5, 1], [6, 0], [7, 10], [8, 9]]
[[9, 9], [10, 8], [0, 7], [1, 6], [2, 5], [3, 4]]
[[4, 4], [5, 3], [6, 2], [7, 1], [8, 0], [9, 10]]
[[10, 10], [0, 9], [1, 8], [2, 7], [3, 6], [4, 5]]
[[5, 5], [6, 4], [7, 3], [8, 2], [9, 1], [10, 0]]
[[0, 0], [10, 1], [9, 2], [8, 3], [7, 4], [6, 5]]
[[6, 6], [5, 7], [4, 8], [3, 9], [2, 10], [1, 0]]
[[1, 1], [0, 2], [10, 3], [9, 4], [8, 5], [7, 6]]
[[7, 7], [6, 8], [5, 9], [4, 10], [3, 0], [2, 1]]
[[2, 2], [1, 3], [0, 4], [10, 5], [9, 6], [8, 7]]
[[8, 8], [7, 9], [6, 10], [5, 0], [4, 1], [3, 2]]
[[3, 3], [2, 4], [1, 5], [0, 6], [10, 7], [9, 8]]
[[9, 9], [8, 10], [7, 0], [6, 1], [5, 2], [4, 3]]
[[4, 4], [3, 5], [2, 6], [1, 7], [0, 8], [10, 9]]

In [153]:
two_q_fidelity = np.zeros((n,n))
for round in table:
    c = Circuit()
    for a,b in round:
        if(a == b):
            c.i(a)
            continue
        c.h(a)
        c.cnot(a,b)
    c = noise.apply(c)

    shotnum = 10000
    task = device.run(c, shots = shotnum)
    result = task.result()
    measurement = result.measurement_counts
    measurement = {int(state[::-1], 2) : cnt for state, cnt in measurement.items()}
    
    for a,b in round:
        if(a == b): continue
        cur_cnt = 0
        for state, cnt in measurement.items():
            if state&(1<<a) and state&(1<<b):
                cur_cnt += cnt
            if (not state&(1<<a)) and (not state&(1<<b)):
                cur_cnt += cnt
        two_q_fidelity[a][b] = cur_cnt/shotnum

print(two_q_fidelity, sep="\n")
        

[[0.     0.9598 0.9628 0.9598 0.9616 0.9602 0.9585 0.9601 0.9682 0.9623
  0.9614]
 [0.9626 0.     0.9657 0.9598 0.9564 0.9648 0.9603 0.9608 0.9598 0.9635
  0.9661]
 [0.9632 0.9604 0.     0.9609 0.968  0.9632 0.9573 0.9684 0.9598 0.9629
  0.9646]
 [0.96   0.9627 0.9641 0.     0.9634 0.9627 0.9696 0.968  0.9627 0.9637
  0.9651]
 [0.9664 0.9652 0.9663 0.9642 0.     0.9611 0.9648 0.9678 0.9652 0.9651
  0.9601]
 [0.9617 0.9596 0.9648 0.9617 0.9595 0.     0.9641 0.9638 0.9619 0.9588
  0.9581]
 [0.9635 0.9668 0.9696 0.9616 0.9667 0.9613 0.     0.9622 0.9608 0.9603
  0.9675]
 [0.963  0.9626 0.9659 0.9648 0.9636 0.966  0.9669 0.     0.9694 0.9596
  0.959 ]
 [0.961  0.9594 0.9655 0.9684 0.9676 0.964  0.9622 0.9635 0.     0.9596
  0.9616]
 [0.9635 0.9637 0.9597 0.9605 0.9583 0.959  0.968  0.9602 0.9592 0.
  0.9636]
 [0.9645 0.9661 0.9611 0.9633 0.9713 0.9671 0.9676 0.9612 0.9615 0.9629
  0.    ]]
